In [1]:
dataset = Edge2Shoes("/mnt/c/Users/Public/Documents/Datasets/edge2shoes", batch_size, 32, device="cuda:0", split="train")
data_loader = torch.utils.data.DataLoader(dataset,
                                               batch_size=batch_size,
                                               shuffle=False,
                                               num_workers=4,
                                               pin_memory=True,
                                               sampler=train_sampler,
                                               drop_last = True)

NameError: name 'Edge2Shoes' is not defined